In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!git clone https://<github_token>@github.com/Thomas-Kragerud/EECS282-Final-Project.git
%cd EECS282-Final-Project/diffusers
!git pull
!git checkout devMorten
#!mkdir out

Cloning into 'EECS282-Final-Project'...
remote: Enumerating objects: 3038, done.
remote: Counting objects: 100% (3038/3038), done.
remote: Compressing objects: 100% (2409/2409), done.
remote: Total 3038 (delta 575), reused 3036 (delta 573), pack-reused 0
Receiving objects: 100% (3038/3038), 15.32 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (575/575), done.
/content/EECS282-Final-Project/diffusers/examples/dreambooth/EECS282-Final-Project/diffusers
Already up to date.
Branch 'devMorten' set up to track remote branch 'devMorten' from 'origin'.
Switched to a new branch 'devMorten'


In [ ]:
!pip install -e .


Obtaining file:///content/EECS282-Final-Project/diffusers/examples/dreambooth/EECS282-Final-Project/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-0.editable-py3-none-any.whl size=11051 sha256=f204b522125263814c04fd52cd39b8ad04578e2dbd228aecdd777f86040f3294
  Stored in directory: /tmp/pip-ephem-wheel-cache-j0htowfa/wheels/23/c3/2f/e82406bbefd381cde1da615e9e2777960f43900f783ab05f0a
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.24.0.dev0
    Uninstalling diffusers-0.24.0.dev0:
      Successfully uninstalled diffusers-0.24.0.dev0


In [ ]:
!git pull

Already up to date.


In [ ]:
%cd examples/dreambooth
!pip install -r requirements.txt
!pip install wandb
!pip install pytorch-ignite
!pip install clean-fid

# Some temp bugfix from github


# !pip install accelerate
# !pip install torchvision
# !pip install transformers
# !pip install ftfy
# !pip install tensorboard
# !pip install Jinja2

/content/EECS282-Final-Project/diffusers/examples/dreambooth
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
!accelerate config default


accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

alvan-nee-brFsZ7qszSY-unsplash.jpeg:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

alvan-nee-bQaAJCbNq3g-unsplash.jpeg:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

alvan-nee-Id1DBHv4fbg-unsplash.jpeg:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

alvan-nee-eoqnr8ikwFE-unsplash.jpeg:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

alvan-nee-9M0tSjb-cpA-unsplash.jpeg:   0%|          | 0.00/677k [00:00<?, ?B/s]

'/content/EECS282-Final-Project/diffusers/examples/dreambooth/dog'

In [ ]:
!mkdir out

In [ ]:
import os

os.environ['MODEL_NAME'] = "runwayml/stable-diffusion-v1-5"
os.environ['INSTANCE_DIR'] = "dog"
os.environ['OUTPUT_DIR'] = "out"  # Change this to your desired output directory.

!huggingface-cli login

!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --instance_validation_dir="instance_validation_dir" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=1 \
  --seed="0" \
  --with_prior_preservation \
  --prior_loss_weight=1 \
  --num_class_images=100 \
  --num_validation_images=100 \
  --class_data_dir="class_data_dir" \
  --class_prompt="a photo of dog" \
  --class_validation_dir="class_validation_dir" \
  --validation_data_dir="validation_images" \

# Saves images every 5 epochs, en epoch er 5 steg, så hvert 5*5 steg lagres noe


# !accelerate launch train_dreambooth.py \
#   --pretrained_model_name_or_path=$MODEL_NAME  \
#   --instance_data_dir=$INSTANCE_DIR \
#   --output_dir=$OUTPUT_DIR \
#   --instance_prompt="a photo of sks dog" \
#   --resolution=512 \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=5e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --max_train_steps=400


/usr/bin/python3: can't open file '/content/EECS282-Final-Project/diffusers/examples/dreambooth/EECS282-Final-Project/diffusers/train_dreambooth_lora.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 47, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 994, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 636, in simple_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/usr/bin/python3', 'train_dreambooth_lora.py', '--pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5', '--instance_data_dir=dog', '--output_dir=out', '--instance_prompt=a photo of sks dog', '--inst

In [ ]:
#%cd out

!zip -r out.zip out


  adding: out/ (stored 0%)


In [ ]:
from google.colab import files
files.download('out.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#!pip install diffusers

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

ModuleNotFoundError: ignored

In [ ]:
pipe.unet.load_attn_procs("patrickvonplaten/lora_dreambooth_dog_example")